In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
!pip install --upgrade transformers datasets accelerate evaluate -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset

# --------------------
# CONFIG
# --------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

MODELS = [
    "bert-base-uncased",
    "roberta-base",
    "microsoft/deberta-v3-base",
    "google/electra-base-discriminator",
    "distilbert-base-uncased"
]
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5

# --------------------
# LOAD DATA (Kaggle Path)
# --------------------
print("Loading data...")
data_path = "/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv"
df = pd.read_csv(data_path)
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# --------------------
# TRAIN/VAL/TEST SPLIT (Subset for model comparison)
# --------------------
# Use smaller subset (e.g. 4000) for faster runs if needed
df_subset, _ = train_test_split(df, train_size=8000, stratify=df['label'], random_state=SEED)
train_df, temp_df = train_test_split(df_subset, train_size=0.7, stratify=df_subset['label'], random_state=SEED)
val_df, test_df = train_test_split(temp_df, train_size=0.5, stratify=temp_df['label'], random_state=SEED)

# Convert to Dataset objects
def df_to_dataset(dataframe, tokenizer):
    return Dataset.from_pandas(dataframe).map(
        lambda x: tokenizer(x['review'], truncation=True, padding="max_length", max_length=MAX_LENGTH),
        batched=True
    )

# --------------------
# METRICS FUNCTION
# --------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average='macro')
    rec = recall_score(labels, preds, average='macro')
    return {"f1": f1, "accuracy": acc, "precision": prec, "recall": rec}

# --------------------
# TRAINING LOOP FOR SUBSET EXPERIMENTS
# --------------------
results = []
for model_name in MODELS:
    print(f"\n=== Training {model_name} on subset ===")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    train_dataset = df_to_dataset(train_df, tokenizer)
    val_dataset = df_to_dataset(val_df, tokenizer)

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    training_args = TrainingArguments(
        output_dir=f"./results/{model_name.replace('/', '_')}",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,   # avoid disk overflow
        learning_rate=LR,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        seed=SEED,
        logging_dir=f"./logs/{model_name.replace('/', '_')}",
        logging_steps=50,
        report_to="none",
        fp16=True,  # mixed precision for faster GPU training
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        processing_class=tokenizer,  # replaces deprecated tokenizer arg
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate()
    results.append({"model": model_name, "f1": metrics["eval_f1"], "accuracy": metrics["eval_accuracy"]})

# --------------------
# SELECT BEST MODEL
# --------------------
results_df = pd.DataFrame(results)
print("\n=== Subset Results ===")
print(results_df)
best_model_name = results_df.sort_values(by="f1", ascending=False).iloc[0]['model']
print(f"\nBest model based on F1: {best_model_name}")

# --------------------
# FULL DATA TRAINING
# --------------------
print("\n=== Training best model on full dataset ===")
tokenizer = AutoTokenizer.from_pretrained(best_model_name)

full_train_df, full_test_df = train_test_split(df, train_size=0.8, stratify=df['label'], random_state=SEED)
full_train_dataset = df_to_dataset(full_train_df, tokenizer)
full_test_dataset = df_to_dataset(full_test_df, tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(best_model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir=f"./final/{best_model_name.replace('/', '_')}",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,   # limit checkpoints
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    seed=SEED,
    logging_dir=f"./logs/final_{best_model_name.replace('/', '_')}",
    logging_steps=50,
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_test_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
final_metrics = trainer.evaluate()
print("\n=== Final Model Evaluation on Test Set ===")
print(final_metrics)

# --------------------
# INFERENCE ON 10 RANDOM TEST REVIEWS
# --------------------
print("\n=== Inference on 10 random samples ===")
sample_df = full_test_df.sample(10, random_state=SEED)
sample_dataset = Dataset.from_pandas(sample_df).map(
    lambda x: tokenizer(x['review'], truncation=True, padding="max_length", max_length=MAX_LENGTH),
    batched=True
)

preds = trainer.predict(sample_dataset)
pred_labels = np.argmax(preds.predictions, axis=-1)
pred_probs = torch.nn.functional.softmax(torch.tensor(preds.predictions), dim=-1).numpy()

for i, row in enumerate(sample_df.itertuples()):
    print(f"\nReview: {row.review[:300]}...")
    print(f"True Label: {row.label} | Predicted: {pred_labels[i]} | Confidence: {pred_probs[i][pred_labels[i]]:.4f}")


2025-09-07 16:06:00.153612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757261160.468927      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757261160.562569      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading data...

=== Training bert-base-uncased on subset ===


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



=== Final Model Evaluation on Test Set ===
{'eval_loss': 0.6979074478149414, 'eval_f1': 0.9495962721402875, 'eval_accuracy': 0.9496, 'eval_precision': 0.9497330490252236, 'eval_recall': 0.9496, 'eval_runtime': 128.9317, 'eval_samples_per_second': 77.56, 'eval_steps_per_second': 2.428, 'epoch': 3.0}

=== Inference on 10 random samples ===


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Review: This is loosely based on the ideas of the original 80's hit . It's set in the modern day as we see a base in Afghanistan get destroyed by a UAV right at the start.<br /><br />And that's exactly where the movie jumps the shark. UAV's aren't armed. They could be but I don't think it's ever been tried ...
True Label: 0 | Predicted: 0 | Confidence: 1.0000

Review: This movie was on British TV last night, and is wonderful! Strong women, great music (most of the time) and just makes you think. We do have stereotypes of what older people "ought" to do, and there are fantastic cameos of the "sensible but worried children". Getting near to my best movie ever !...
True Label: 1 | Predicted: 1 | Confidence: 1.0000

Review: What a dreadful movie. The effects were poor, especially by todays standards, but that was forgivable. What was unforgivable was the terrible rehashing of every flood/dam breaks disaster movie ever made into this piece of trash. The acting was awful and I mean AWFUL. T